# Reuters-21578 collection

Reuters-21578 is arguably the most commonly used collection for text classification during the last two decades.
It has been used in some of the most influential papers on the field. 

This dataset contains structured information about newswire articles that can be assigned to several classes, 
therefore making this a multi-label problem. It has a highly skewed distribution of documents over categories, 
where a large proportion of documents belong to few topics.

The collection originally consists of 21,578 documents, including documents without topics and typographical errors. 
For this reason, a subset and split of the collection is traditionally used. This split also focus only on the 
categories that have at least one document in the training set and the test set. The dataset has 90 categories with a 
training set of 7769 documents and a test set of 3019 documents.


# Data Collection Stats

Understanding your collection is always the first step of any data science problem. Lets have a quick look at the Reuters collection and its documents.

In [2]:
from nltk.corpus import reuters

# List of document ids
documents = reuters.fileids()
print("Documents: {}".format(len(documents)))

# Train documents
train_docs_id = list(filter(lambda doc: doc.startswith("train"),documents))
print("Total train documents: {}".format(len(train_docs_id)))

# Test documents
test_docs_id = list(filter(lambda doc: doc.startswith("test"),documents))
print("Total test documents: {}".format(len(test_docs_id)))

Documents: 10788
Total train documents: 7769
Total test documents: 3019


In [3]:
# Example of a document (with multiple labels)
doc = 'training/9865'
print(reuters.raw(doc))
print()
print(reuters.categories(doc))

FRENCH FREE MARKET CEREAL EXPORT BIDS DETAILED
  French operators have requested licences
  to export 675,500 tonnes of maize, 245,000 tonnes of barley,
  22,000 tonnes of soft bread wheat and 20,000 tonnes of feed
  wheat at today's European Community tender, traders said.
      Rebates requested ranged from 127.75 to 132.50 European
  Currency Units a tonne for maize, 136.00 to 141.00 Ecus a tonne
  for barley and 134.25 to 141.81 Ecus for bread wheat, while
  rebates requested for feed wheat were 137.65 Ecus, they said.
  



['barley', 'corn', 'grain', 'wheat']


In [4]:
from operator import itemgetter
from pprint import pprint

In [5]:
# List of categories
categories = reuters.categories()
print("Number of categories: {}".format(len(categories)))
print()
print(categories)

Number of categories: 90

['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


In [6]:
# Documents per category
category_distribution = [(category, len(reuters.fileids(category)))
                        for category in categories]
category_distribution = sorted(category_distribution,key = itemgetter(1),reverse = True)

print("Most common categories")
pprint(category_distribution[:5])
print()
print("Least common categories")
pprint(category_distribution[-5:])
print()

Most common categories
[('earn', 3964),
 ('acq', 2369),
 ('money-fx', 717),
 ('grain', 582),
 ('crude', 578)]

Least common categories
[('castor-oil', 2),
 ('groundnut-oil', 2),
 ('lin-oil', 2),
 ('rye', 2),
 ('sun-meal', 2)]



In [7]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

In [8]:
stop_words = stopwords.words("english")

In [9]:
train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]

# Tokenisation

In [11]:
vectorizer = TfidfVectorizer(stop_words=stop_words)

# Learn and transform train documents
vectorised_train_documents = vectorizer.fit_transform(train_docs)
vectorised_test_documents = vectorizer.transform(test_docs)

# Transform multilabel labels

In [12]:
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(reuters.categories(doc_id) 
                                for doc_id in train_docs_id)
test_labels = mlb.transform(reuters.categories(doc_id)
                           for doc_id in test_docs_id)

# Classifier

In [13]:
classifier = OneVsRestClassifier(LinearSVC(random_state=42))
classifier.fit(vectorised_train_documents,train_labels)
predictions = classifier.predict(vectorised_test_documents)

print("Numbers of labels assigned: {}".format(sum([sum(prediction)
                                                  for prediction in predictions])))

Numbers of labels assigned: 3126


# How well have we done?

In [14]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Micro-average scores

In [15]:
precision = precision_score(test_labels, predictions, average = 'micro')
recall = recall_score(test_labels, predictions, average = 'micro')
f1 = f1_score(test_labels, predictions, average = 'micro')
print("Micro-average quality numbers")
print("Precision: {}, Recall: {}, F1-measure: {}".format(precision,recall,f1))

Micro-average quality numbers
Precision: 0.9516954574536148, Recall: 0.7946047008547008, F1-measure: 0.8660844250363902


In [16]:
precision = precision_score(test_labels, predictions, average='macro')
recall = recall_score(test_labels, predictions, average='macro')
f1 = f1_score(test_labels, predictions, average='macro')
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, 
                                                                     recall, 
                                                                     f1))

Macro-average quality numbers
Precision: 0.6305, Recall: 0.3715, F1-measure: 0.4451


/opt/anaconda/envs/Python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda/envs/Python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## What is this weird warning message about "Precision being ill-defined?"

Some metrics can be in a position of indeterminate value when for instance, the classifier decides to not classify any articles in a specific category. This would imply a 0/0 precision. Scikit learn warns us about this, and reports back this quality as 0.0.